In [1]:
import pandas as pd
from datetime import datetime, timedelta
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import random
import os

In [2]:
# Obtener el directorio actual de trabajo
current_dir = os.getcwd()

# Subir un nivel desde notebooks para llegar a la carpeta raíz del proyecto
project_root = os.path.dirname(current_dir)

# Construir rutas relativas para entrada y salida
input_path = os.path.join(project_root, 'data', 'banksim.csv')

output_path = os.path.join(project_root, 'data', 'processed_banksim.csv')

# Leer el CSV
df = pd.read_csv(input_path)

In [3]:
df['fraud'] = df['fraud'].astype(int)
df['amount'] = df['amount'].astype(float)
df['step'] = df['step'].astype(int)
df['hour'] = df['step'] % 24
df['date'] = pd.to_datetime('2024-01-01') + pd.to_timedelta(df['step'], 'D')
# Convertir 'step' a fechas empezando desde 2024-01-01
df['date'] = pd.to_datetime('2024-01-01') + pd.to_timedelta(df['step'], unit='D')


df['date_formatted'] = df['date'].dt.strftime('%Y-%m-%d')

df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['weekday'] = df['date'].dt.day_name()

# List of random European postal codes (sample)
european_postal_codes = [
    '10115', '75008', '00185', '28013', '69002', '1000', '20095', '11000', '5020', '8001'
]

# Function to replace postal codes randomly
def replace_postal_codes(df, column_name, postal_code_list):
    df[column_name] = df[column_name].apply(lambda x: random.choice(postal_code_list))
    return df

# Replace postal codes in `zipcodeOri` and `zipMerchant` columns
df['zipcodeOri'] = df['zipcodeOri'].str.strip("'")  # Remove quotes around the data
df['zipMerchant'] = df['zipMerchant'].str.strip("'")

df = replace_postal_codes(df, 'zipcodeOri', european_postal_codes)
df = replace_postal_codes(df, 'zipMerchant', european_postal_codes)

# Función para geocodificar códigos postales
def get_location_info(zipcode, country='ES'):  # Por defecto España, ajustar según necesidad
    geolocator = Nominatim(user_agent="fraud_dashboard")
    try:
        location = geolocator.geocode(f"{zipcode}, {country}")
        if location:
            return {
                'lat': location.latitude,
                'lon': location.longitude,
                'address': location.address
            }
    except GeocoderTimedOut:
        return None
    return None
# Función para extraer el barrio de la dirección
def extract_neighborhood(address):
    if pd.isna(address):
        return None
    
    # Dividir la dirección en partes
    parts = address.split(',')
    
    # Generalmente, el barrio suele estar en una de las primeras partes de la dirección
    for part in parts:
        # Eliminar espacios al inicio y al final
        part = part.strip()
        
        # Criterios para identificar un barrio (puedes ajustarlos según necesites)
        if len(part) > 2 and len(part) < 30:
            return part
    
    return None


# Crear cache de geocodificación
zipcode_cache = {}
unique_zipcodes = pd.concat([df['zipcodeOri'], df['zipMerchant']]).unique()

for zipcode in unique_zipcodes:
    if str(zipcode) not in zipcode_cache:
        location_info = get_location_info(str(zipcode))
        if location_info:
            zipcode_cache[str(zipcode)] = location_info

# Agregar información de ubicación al DataFrame
df['origin_lat'] = df['zipcodeOri'].map(lambda x: zipcode_cache.get(str(x), {}).get('lat'))
df['origin_lon'] = df['zipcodeOri'].map(lambda x: zipcode_cache.get(str(x), {}).get('lon'))
df['origin_address'] = df['zipcodeOri'].map(lambda x: zipcode_cache.get(str(x), {}).get('address'))
df['merchant_lat'] = df['zipMerchant'].map(lambda x: zipcode_cache.get(str(x), {}).get('lat'))
df['merchant_lon'] = df['zipMerchant'].map(lambda x: zipcode_cache.get(str(x), {}).get('lon'))
df['merchant_address'] = df['zipMerchant'].map(lambda x: zipcode_cache.get(str(x), {}).get('address'))
# Agregar columnas de barrio
df['origin_neighborhood'] = df['origin_address'].apply(extract_neighborhood)
df['merchant_neighborhood'] = df['merchant_address'].apply(extract_neighborhood)


In [4]:
df.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud,...,day,weekday,origin_lat,origin_lon,origin_address,merchant_lat,merchant_lon,merchant_address,origin_neighborhood,merchant_neighborhood
0,0,'C1093826151','4','M',5020,'M348934600',11000,'es_transportation',4.55,0,...,1,Monday,-20.306962,-40.294351,"5020, Avenida Desembargador Santos Neves, Sant...",42.963310,-1.619653,"11000, Errotaldea kalea, Olague, Egintto, Olag...",5020,11000
1,0,'C352968107','2','M',20095,'M348934600',28013,'es_transportation',39.68,0,...,1,Monday,42.181415,-1.605458,"20095, Travesía de Arguedas - Valtierra, Argue...",40.418766,-3.707066,"28013, Centro, Madrid, Comunidad de Madrid, Es...",20095,28013
2,0,'C2054744914','4','F',5020,'M1823072687',5020,'es_transportation',26.89,0,...,1,Monday,-20.306962,-40.294351,"5020, Avenida Desembargador Santos Neves, Sant...",-20.306962,-40.294351,"5020, Avenida Desembargador Santos Neves, Sant...",5020,5020
3,0,'C1760612790','3','M',69002,'M348934600',8001,'es_transportation',17.25,0,...,1,Monday,47.876537,16.531285,"ES, Fürstenweg, Stotzing, Bezirk Eisenstadt-Um...",42.784503,-2.119794,"8001, Calle de la Fuente, Baquedano / Bakedao,...",Fürstenweg,8001
4,0,'C757503768','5','M',10115,'M348934600',69002,'es_transportation',35.72,0,...,1,Monday,52.500363,13.300650,"ES, Damaschkestraße, Charlottenburg, Charlotte...",47.876537,16.531285,"ES, Fürstenweg, Stotzing, Bezirk Eisenstadt-Um...",Damaschkestraße,Fürstenweg


In [5]:
# Exportar el DataFrame procesado a CSV
df.to_csv(output_path, index=False)

print("DataFrame exportado exitosamente a processed_banksim.csv")

DataFrame exportado exitosamente a processed_banksim.csv
